## Quarterbacks

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from fantasy_football import Fantasy

# Import fantasy class which will be used for building datasets
football = Fantasy()

First, we pull 2022 offense data and clean it

In [13]:
df, y = Fantasy.read_data('2021')

TypeError: unsupported operand type(s) for -: 'str' and 'int'

In [12]:
# df_year_2, y_2 = Fantasy.read_data('2022')

Split data by positional group

In [4]:
QB_data = football.makeRegularData(df, "QB")


Next, we pull more advanced data for rushing and passing

In [5]:
# Passing advanced
passing_advanced = football.getAdvancedStats("passing", "2021")
passing_advanced

,Rk,Player,Tm,Age,Pos,G,GS,Cmp,Att,Yds,IAY,IAY/PA,CAY,CAY/Cmp,CAY/PA,YAC,YAC/Cmp
0,1,Tom Brady,TAM,44,QB,17,17,485,719,5316,5791,8.1,2762,5.7,3.8,2554,5.3
1,2,Justin Herbert,LAC,23,QB,17,17,443,672,5014,5076,7.6,2627,5.9,3.9,2387,5.4
2,3,Matthew Stafford,LAR,33,QB,17,17,404,601,4886,5092,8.5,2760,6.8,4.6,2126,5.3
3,4,Patrick Mahomes,KAN,26,QB,17,17,436,658,4839,4829,7.3,2140,4.9,3.3,2699,6.2
4,5,Derek Carr,LVR,30,QB,17,17,428,626,4804,5087,8.1,2525,5.9,4.0,2279,5.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110,111,Cordarrelle Patterson,ATL,30,RB,16,13,0,1,0,18,18.0,0,NaN,0.0,0,NaN
111,112,Brett Rypien,DEN,25,QB,1,0,0,2,0,11,5.5,0,NaN,0.0,0,NaN
112,113,D'Andre Swift,DET,22,RB,13,4,0,1,0,-12,-12.0,0,NaN,0.0,0,NaN
113,114,Albert Wilson,MIA,29,WR,14,5,0,1,0,5,5.0,0,NaN,0.0,0,NaN


Now we are ready to merge the datasets to incorporate the more advance statistics

In [6]:
QB = football.addAdvanced(QB_data, passing_advanced)

0          T. Brady
1        J. Herbert
2       M. Stafford
3        P. Mahomes
4           D. Carr
           ...     
110    C. Patterson
111       B. Rypien
112        D. Swift
113       A. Wilson
114      B. Zylstra
Name: Name, Length: 115, dtype: object
6             L. Jackson
12              J. Allen
16             R. Wilson
19             K. Murray
26            P. Mahomes
29           D. Prescott
34            J. Herbert
45              T. Brady
46            A. Rodgers
59          R. Tannehill
63           M. Stafford
64              J. Hurts
88             J. Burrow
102          T. Lawrence
106              M. Ryan
109           K. Cousins
110          B. Mayfield
127            D. Watson
130       R. Fitzpatrick
132             T. Lance
133            J. Fields
146    B. Roethlisberger
148             D. Jones
151        T. Tagovailoa
154              D. Carr
164           J. Winston
167             C. Wentz
173            Z. Wilson
179           S. Darnold
186            C

In [8]:
QB

,Player_x,Pos_x,Tm_x,GP,Comp,Inc,Passing_Yds,Passing_Td,Int,Pic6,...,Cmp,Att_y,Yds,IAY,IAY/PA,CAY,CAY/Cmp,CAY/PA,YAC,YAC/Cmp
0,L. Jackson,QB,Bal,15,242,134,2757,26,9,1,...,246.0,382.0,2882.0,3543.0,9.3,1734.0,7.0,4.5,1148.0,4.7
1,J. Allen,QB,Buf,16,396,176,4544,37,10,0,...,409.0,646.0,4407.0,5304.0,8.2,2664.0,6.5,4.1,1743.0,4.3
2,R. Wilson,QB,Sea,16,384,174,4212,40,13,1,...,259.0,400.0,3113.0,3956.0,9.9,1730.0,6.7,4.3,1383.0,5.3
3,K. Murray,QB,Ari,16,375,183,3971,26,12,1,...,333.0,481.0,3787.0,3847.0,8.0,1986.0,6.0,4.1,1801.0,5.4
4,P. Mahomes,QB,KC,15,390,198,4740,38,6,0,...,436.0,658.0,4839.0,4829.0,7.3,2140.0,4.9,3.3,2699.0,6.2
5,D. Prescott,QB,Dal,5,151,71,1856,9,4,1,...,410.0,596.0,4449.0,4592.0,7.7,2425.0,5.9,4.1,2024.0,4.9
6,J. Herbert,QB,LAC,15,396,199,4336,31,10,0,...,443.0,672.0,5014.0,5076.0,7.6,2627.0,5.9,3.9,2387.0,5.4
7,T. Brady,QB,TB,16,401,209,4633,40,12,2,...,485.0,719.0,5316.0,5791.0,8.1,2762.0,5.7,3.8,2554.0,5.3
8,A. Rodgers,QB,GB,16,372,154,4299,48,5,1,...,366.0,531.0,4115.0,4099.0,7.7,1947.0,5.3,3.7,2168.0,5.9
9,R. Tannehill,QB,Ten,16,315,166,3819,33,7,0,...,357.0,531.0,3734.0,3959.0,7.5,1958.0,5.5,3.7,1776.0,5.0


### CONTRACTS AND OTHER DATA (not yet completed)

We are not using the player contracts yet but here is where we scrape that data

In [7]:
table_contracts = pd.read_html('https://www.spotrac.com/nfl/contracts/sort-value/offense/all-time/free-agent-2021/limit-100/')

player_contracts = table_contracts[0]
player_contracts

,Rank,Player,Signed Age,Yrs,Value,AAV,Sign Bonus,G'teed @ Sign,Practical G'teed
0,1,Jay Cutler Quarterback | 2014-2020 (FA: 2021)...,30,7,"$126,700,000","$18,100,000",NaN,"$38,000,000","$54,000,000"
1,2,Cam Newton Quarterback | 2015-2020 (FA: 2021),26,5,"$103,800,000","$20,760,000","$22,500,000","$41,000,000","$60,000,000"
2,3,Andy Dalton Quarterback | 2014-2020 (FA: 2021...,26,6,"$96,000,000","$16,000,000","$12,000,000","$17,000,000","$17,000,000"
3,4,Kirk Cousins Quarterback | 2018-2020 (FA: 2021),29,3,"$84,000,000","$28,000,000","$3,000,000","$84,000,000","$84,000,000"
4,5,Ryan Tannehill Quarterback | 2015-2020 (FA: 2...,26,4,"$77,000,000","$19,250,000","$11,500,000","$21,500,000","$45,000,000"
...,...,...,...,...,...,...,...,...,...
95,96,Paxton Lynch Quarterback | 2016-2020 (FA: 202...,22,4,"$9,476,296","$2,369,074","$5,091,852","$7,884,074","$7,884,074"
96,97,Lee Smith Tight End | 2018-2020 (FA: 2021) O...,30,3,"$9,000,000","$3,000,000",NaN,"$3,000,000","$3,000,000"
97,97,Sammy Watkins Wide Receiver | 2020-2020 (FA: ...,26,1,"$9,000,000","$9,000,000","$1,000,000","$8,000,000","$8,000,000"
98,97,Donte Moncrief Wide Receiver | 2019-2020 (FA:...,25,2,"$9,000,000","$4,500,000","$3,500,000","$3,500,000","$3,500,000"
